In [ ]:
#Installing libraries for the first run
!pip install datasets
!pip install transformers
!pip install --upgrade accelerate
#accelerate is bugged this is necessary
!pip uninstall -y transformers accelerate
!pip install transformers accelerate
!pip install evaluate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading the datasets

In [ ]:
import pandas as pd

path = "/content/drive/My Drive/Colab Notebooks/train_dev_test_splits"

dataset = {}
dataset["de"] = {}
dataset["fr"] = {}

dataset["de"]["train"] = pd.read_csv(path + "/de.train.csv", sep="\t")
dataset["de"]["val"] = pd.read_csv(path + "/de.valid.csv", sep="\t")
dataset["de"]["test"] = pd.read_csv(path + "/de.test.csv", sep="\t")

dataset["fr"]["train"] = pd.read_csv(path + "/fr.train.csv", sep="\t")
dataset["fr"]["val"] = pd.read_csv(path + "/fr.valid.csv", sep="\t")
dataset["fr"]["test"] = pd.read_csv(path + "/fr.test.csv", sep="\t")

Transform Pandas DataFrames into HugginFace Datasets

In [ ]:
def prepare_dataset_task(lang_dataset, task):
  drop_cols = list(lang_dataset["train"].columns)
  drop_cols.remove("content")
  drop_cols.remove(task)
  for split in ["train", "val", "test"]:
    lang_dataset[split].drop(columns=drop_cols, inplace=True)
    lang_dataset[split].rename(columns={task :'label'}, inplace = True)
  return lang_dataset
  

In [ ]:
from datasets import Dataset, DatasetDict

def create_dict(lang_dataset, other_dataset=None):
  if other_dataset:
    for split in ["train", "val"]:
      lang_dataset[split] = Dataset.from_pandas(pd.concat([lang_dataset[split], other_dataset[split]]))
    return DatasetDict({
      'train': lang_dataset["train"],
      'validation': lang_dataset["val"],
      'test_fr': Dataset.from_pandas(lang_dataset["test"]),
      'test_de': Dataset.from_pandas(other_dataset["test"])
    })
  else:
    for split in ["train", "val", "test"]:
      lang_dataset[split] = Dataset.from_pandas(lang_dataset[split])
    # gather everyone if you want to have a single DatasetDict
    return DatasetDict({
      'train': lang_dataset["train"],
      'validation': lang_dataset["val"],
      'test': lang_dataset["test"]
    })

In [ ]:
#!pip install transformers
from transformers import AutoTokenizer

def tokenize_datset(model_name, dataset_dict):
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  def preprocess_function(sample):
    return tokenizer(sample["content"], truncation=True)

  return dataset_dict.map(preprocess_function, batched=True), tokenizer
  

In [ ]:
#!pip install --upgrade accelerate
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from datasets import load_metric
import numpy as np

def create_trainer(model_name, tokenized_dataset, tokenizer, results_folder):
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

  metric = load_metric("f1")

  def compute_metrics(eval_pred):
      logits, labels = eval_pred
      predictions = np.argmax(logits, axis=-1)
      return metric.compute(predictions=predictions, references=labels, average="macro")

  training_args = TrainingArguments(
      output_dir=f"./{results_folder}",
      learning_rate=2e-5,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      num_train_epochs=20,
      weight_decay=0.01,
      evaluation_strategy="epoch", # run validation at the end of each epoch
      save_strategy="epoch",
      load_best_model_at_end=True,
  )

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  return Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_dataset["train"],
      eval_dataset=tokenized_dataset["validation"],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics
  )

In [ ]:
import numpy as np
import evaluate

def evaluation(predictions):
  preds = np.argmax(predictions.predictions, axis=-1)
  metric = evaluate.load("f1")
  print(metric.compute(predictions=preds, references=predictions.label_ids, average="macro"))


In [ ]:
from copy import deepcopy

def train_test(lang, task, model_name):
  #Load data and prepare for task
  if lang != "concat":
    lang_dataset = deepcopy(dataset[lang])
    lang_dataset_task = prepare_dataset_task(lang_dataset, task)
    train_valid_test_dataset = create_dict(lang_dataset_task)
  else:
    dataset_fr = deepcopy(dataset["fr"])
    dataset_de = deepcopy(dataset["de"])
    dataset_task_fr = prepare_dataset_task(dataset_fr, task)
    dataset_task_de = prepare_dataset_task(dataset_de, task)
    train_valid_test_dataset = create_dict(dataset_fr, dataset_de)
  
  #Tokenize dataset
  tokenized_dataset, tokenizer = tokenize_datset(model_name, train_valid_test_dataset)
  print(model_name)
  #Training
  results_folder = f"{lang}-{task}-{model_name}"
  trainer = create_trainer(model_name, tokenized_dataset, tokenizer, results_folder)
  trainer.train()
  trainer.save_model()

  #Evaluation
  if lang != "concat":
    predictions = trainer.predict(test_dataset=tokenized_dataset["test"])
    evaluation(predictions)
  else:
    predictions_fr = trainer.predict(test_dataset=tokenized_dataset["test_fr"])
    predictions_de = trainer.predict(test_dataset=tokenized_dataset["test_de"])
    print("French F1-Macro")
    evaluation(predictions_fr)
    print("German F1-Macro")
    evaluation(predictions_de)


In [ ]:
model_name_de =  "bert-base-german-cased"
model_name_fr = "camembert-base"
model_name_multi = "bert-base-multilingual-cased"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# FRENCH

## E1

In [ ]:
train_test("fr", "e1", model_name_fr)

Map:   0%|          | 0/2178 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight'

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.472012,0.445936
2,No log,0.448514,0.469855
3,No log,0.441297,0.480337
4,0.493400,0.489494,0.464136
5,0.493400,0.587618,0.474641
6,0.493400,0.588257,0.485002
7,0.493400,0.585727,0.612979
8,0.223500,0.637037,0.628754
9,0.223500,0.680200,0.601554
10,0.223500,0.758772,0.609840


{'f1': 0.4650819647942395}


In [ ]:
train_test("fr", "e1", "rodrigotuna/fr-camembert-base-finetuned")

Map:   0%|          | 0/2178 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

rodrigotuna/fr-e1-camembert-base-finetuned


Some weights of the model checkpoint at rodrigotuna/fr-e1-camembert-base-finetuned were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at rodrigotuna/fr-e1-camembert-base-finetuned and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.477067,0.451690
2,No log,0.481955,0.445959
3,No log,0.473875,0.468093
4,0.473500,0.523951,0.467654
5,0.473500,0.582276,0.586042
6,0.473500,0.604030,0.532739
7,0.473500,0.646743,0.631377
8,0.188400,0.665062,0.610363
9,0.188400,0.756205,0.610421
10,0.188400,0.745781,0.615935


{'f1': 0.4593645287429207}


## A1

In [ ]:
train_test("fr", "a1", model_name_fr)

Map:   0%|          | 0/2178 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

camembert-base


Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight'

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.617531,0.401631
2,No log,0.562709,0.690512
3,No log,0.546755,0.703901
4,0.553300,0.657948,0.691809
5,0.553300,0.697944,0.688026
6,0.553300,0.833304,0.709106
7,0.553300,0.968464,0.695833
8,0.227100,0.997946,0.688382
9,0.227100,1.159471,0.700436
10,0.227100,1.207329,0.700566


{'f1': 0.6827276957570116}


In [ ]:
train_test("fr", "a1", "rodrigotuna/fr-camembert-base-finetuned")

Map:   0%|          | 0/2178 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

rodrigotuna/fr-e1-camembert-base-finetuned


Some weights of the model checkpoint at rodrigotuna/fr-e1-camembert-base-finetuned were not used when initializing CamembertForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at rodrigotuna/fr-e1-camembert-base-finetuned and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.655049,0.395405
2,No log,0.571604,0.653500
3,No log,0.571362,0.676794
4,0.572600,0.616650,0.698767
5,0.572600,0.651675,0.714177
6,0.572600,0.721500,0.705470
7,0.572600,0.931121,0.701297
8,0.251800,0.983916,0.712070
9,0.251800,1.074604,0.696181
10,0.251800,1.177189,0.708324


Epoch,Training Loss,Validation Loss,F1
1,No log,0.655049,0.395405
2,No log,0.571604,0.653500
3,No log,0.571362,0.676794
4,0.572600,0.616650,0.698767
5,0.572600,0.651675,0.714177
6,0.572600,0.721500,0.705470
7,0.572600,0.931121,0.701297
8,0.251800,0.983916,0.712070
9,0.251800,1.074604,0.696181
10,0.251800,1.177189,0.708324


{'f1': 0.6628993741765481}


# GERMAN

## E1

In [ ]:
train_test("de", "e1", model_name_de)

Map:   0%|          | 0/2806 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

bert-base-german-cased


Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

Epoch,Training Loss,Validation Loss,F1
1,No log,0.519899,0.472134
2,No log,0.547287,0.480335
3,0.462300,0.834143,0.455351
4,0.462300,1.155043,0.564584
5,0.462300,1.307521,0.513177
6,0.072600,1.474484,0.506919
7,0.072600,1.600919,0.552761
8,0.072600,1.637004,0.548813
9,0.007500,1.685963,0.510558
10,0.007500,1.653322,0.518535


{'f1': 0.4446934764398425}


In [ ]:
train_test("de", "e1", "rodrigotuna/de-bert-base-german-cased-finetuned")

Map:   0%|          | 0/2806 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

rodrigotuna/de-e1-bert-base-german-cased-finetuned


Some weights of the model checkpoint at rodrigotuna/de-e1-bert-base-german-cased-finetuned were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.498161,0.485378
2,No log,0.576300,0.470869
3,0.446000,0.845358,0.526920
4,0.446000,1.198366,0.475978
5,0.446000,1.371479,0.488440
6,0.069100,1.453785,0.526342
7,0.069100,1.591792,0.472581
8,0.069100,1.617959,0.487931
9,0.007600,1.693131,0.537787
10,0.007600,1.655942,0.543942


{'f1': 0.4380821801424111}


## A1

In [ ]:
train_test("de", "a1", model_name_de)

Map:   0%|          | 0/2806 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

bert-base-german-cased


Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

Epoch,Training Loss,Validation Loss,F1
1,No log,0.539973,0.663834
2,No log,0.440382,0.737500
3,0.306200,0.783987,0.733088
4,0.306200,0.927679,0.740174
5,0.306200,1.074828,0.737791
6,0.032400,1.255178,0.733979
7,0.032400,1.359904,0.739448
8,0.032400,1.429906,0.731337
9,0.004900,1.472600,0.752863
10,0.004900,1.511275,0.737500


{'f1': 0.7479645397065431}


In [ ]:
train_test("de", "a1", "rodrigotuna/de-bert-base-german-cased-finetuned")

Map:   0%|          | 0/2806 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

rodrigotuna/de-e1-bert-base-german-cased-finetuned


Some weights of the model checkpoint at rodrigotuna/de-e1-bert-base-german-cased-finetuned were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.493192,0.696443
2,No log,0.446556,0.701897
3,0.299800,0.797728,0.744336
4,0.299800,0.838798,0.748462
5,0.299800,1.073983,0.706537
6,0.035900,1.120460,0.718726
7,0.035900,1.298971,0.738837
8,0.035900,1.427620,0.716640
9,0.003900,1.434423,0.718694
10,0.003900,1.531095,0.728912


{'f1': 0.7358053922453865}


# MULTI-LANGUAGE

## E1

In [ ]:
train_test("concat", "e1", model_name_multi)

Map:   0%|          | 0/4984 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

bert-base-multilingual-cased


...

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.517083,0.425354
2,0.593600,0.570833,0.501211
3,0.593600,0.659724,0.523304
4,0.377900,0.802418,0.524210
5,0.188400,1.049101,0.548728
6,0.188400,1.138928,0.537657
7,0.107400,1.393662,0.537489
8,0.107400,1.615720,0.505469


Epoch,Training Loss,Validation Loss,F1
1,No log,0.517083,0.425354
2,0.593600,0.570833,0.501211
3,0.593600,0.659724,0.523304
4,0.377900,0.802418,0.524210
5,0.188400,1.049101,0.548728
6,0.188400,1.138928,0.537657
7,0.107400,1.393662,0.537489
8,0.107400,1.615720,0.505469
9,0.060900,1.797800,0.533831
10,0.032400,1.658288,0.508438


French F1-Macro


{'f1': 0.4188779333661365}
German F1-Macro
{'f1': 0.38339506929407646}


## A1

In [ ]:
train_test("concat", "a1", model_name_multi)

Map:   0%|          | 0/4984 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

bert-base-multilingual-cased


...

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.511073,0.638216
2,0.501400,0.546026,0.668627
3,0.501400,0.712644,0.670914
4,0.305400,1.196031,0.609289
5,0.137700,1.440562,0.649731
6,0.137700,1.338596,0.670139
7,0.072200,1.732069,0.643833
8,0.072200,1.835283,0.640041
9,0.043700,1.760854,0.650620
10,0.035000,1.580937,0.647366


Epoch,Training Loss,Validation Loss,F1
1,No log,0.511073,0.638216
2,0.501400,0.546026,0.668627
3,0.501400,0.712644,0.670914
4,0.305400,1.196031,0.609289
5,0.137700,1.440562,0.649731
6,0.137700,1.338596,0.670139
7,0.072200,1.732069,0.643833
8,0.072200,1.835283,0.640041
9,0.043700,1.760854,0.650620
10,0.035000,1.580937,0.647366


French F1-Macro


{'f1': 0.6086538879643282}
German F1-Macro
{'f1': 0.6695064001027442}
